In [3]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os
import pandas as pd
import requests
"""
https://stackoverflow.com/questions/74090972/how-to-stream-video-images-from-python-app-to-a-remote-reactjs-webpage
"""

'\nhttps://stackoverflow.com/questions/74090972/how-to-stream-video-images-from-python-app-to-a-remote-reactjs-webpage\n'

In [4]:
model_url = "https://raw.githubusercontent.com/balachander-sa/Namestai_streamlit/main/Movenet_lightning_unit8.tflite"
# Download the TFLite model from GitHub
response = requests.get(model_url)
model_content = response.content

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_content=model_content)
interpreter.allocate_tensors()

In [3]:
"""
[nose,
left eye,
right eye, 
left ear,
right ear, 
left shoulder,
right shoulder,
left elbow, 
right elbow,
left wrist,
right wrist, 
left hip,
right hip,
left knee,
right knee,
left ankle,
right ankle]

"""
"""
1,6,8-------left shoulder angle----LSA
1,7,9-------right shoulder angle---RSA
6,8,10------left elbow angle-------LEA
7,9,11------right elbow angle------REA
6,12,14-----left hip angle---------LHA
7,13,15-----right hip angle--------RHA
12,14,16----left knee angle--------LKA
13,15,17----right knee angle-------RKA
""";

![Movenet reference picture](Movenet_ref.png)

In [12]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame1.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    keypoint_indx = [5,6,7,8,11,12,13,14]

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

    for idx in keypoint_indx:
        kp = shaped[idx]
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 8, (255,0,0),2)

In [10]:
def return_Keypoints(frame, interpreter):

    # Reshape image
    img = frame.copy()
    img = tf.image.resize(np.expand_dims(img, axis=0), [192,192])
    input_image = tf.cast(img, dtype=tf.uint8)

    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    keypoints_final = keypoints_with_scores[0,0].T[0:2]
    draw_keypoints(frame, keypoints_with_scores, 0.01)

    return frame, keypoints_final

In [7]:
def calculate_angle(a,b,c):

    AB = np.array(b) - np.array(a)
    BC = np.array(c) - np.array(b)

    dot_product = np.dot(AB, BC)
    magnitude_AB = np.linalg.norm(AB)
    magnitude_BC = np.linalg.norm(BC)

    cos_theta = dot_product / (magnitude_AB * magnitude_BC)
    theta = np.arccos(cos_theta)

    # Convert to degrees
    angle_degrees = np.degrees(theta)

    return angle_degrees        

In [13]:

folder_path = "Preprocessed_images\Pos1"  #Input folder
Output_folder = "Output_images\Pos1"
# Read Image
Angle_list = []
for filename in os.listdir(folder_path):
    # Read the input image
    image_path = os.path.join(folder_path, filename)
    frame = cv2.imread(image_path)
    frame = cv2.resize(frame, (512,512))

    #Getting the annotated image and keypoints
    op_image, kp = return_Keypoints(frame, interpreter)
    
    #Calculating the required angles
    LSA = calculate_angle(kp[:,0],kp[:,5],kp[:,7])
    RSA = calculate_angle(kp[:,0],kp[:,6],kp[:,8])
    LEA = calculate_angle(kp[:,5],kp[:,7],kp[:,9])
    REA = calculate_angle(kp[:,6],kp[:,8],kp[:,10])
    LHA = calculate_angle(kp[:,5],kp[:,11],kp[:,13])
    RHA = calculate_angle(kp[:,6],kp[:,12],kp[:,14])
    LKA = calculate_angle(kp[:,11],kp[:,13],kp[:,15])
    RKA = calculate_angle(kp[:,12],kp[:,14],kp[:,16])
    Angles = [LSA, RSA, LEA, REA, LHA, RHA, LKA, RKA]

    # Save the output image
    output_image_path = os.path.join(Output_folder, filename)
    cv2.imwrite(output_image_path, frame)

    # Saving the angles to convert in CSV
    Angle_list.append(Angles)

In [13]:
angle_array = np.stack(Angle_list)
angle_array
header_val = ["LSA", "RSA", "LEA", "REA", "LHA", "RHA", "LKA", "RKA"]
format_str = '%.18f'
np.savetxt('Angles_pos1.csv', angle_array, delimiter=',', header=','.join(header_val), fmt=format_str, comments='')

print("CSV file saved successfully.")

CSV file saved successfully.
